In [1]:
import pandas as pd

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.graphs import KG
from pyrdf2vec.walkers import RandomWalker
from pyrdf2vec.graphs import Vertex

import rdflib

In [2]:
knowledge_graph = KG("Query_2020_sortiert_ohneAutor__mitTitel_editiert.ttl", skip_predicates= {"http://prismstandard.org/namespaces/basic/2.0/publicationDate", "http://purl.org/spar/fabio/hasPublicationYear"}, literals=[
    ["http://www.w3.org/2004/02/skos/core#prefLabel"],
])

In [3]:
for subj, pred, obj in rdflib.Graph().parse(
    "Query_2020_sortiert_ohneAutor__mitTitel_editiert.ttl", format="ttl"
):
    subj = Vertex(str(subj))
    obj = Vertex(str(obj))
    pred = Vertex(str(pred), predicate=True, vprev=subj, vnext=obj)
    #knowledge_graph.add_walk(subj, pred, obj)
    knowledge_graph.add_walk(obj, pred, subj)

    # print(subj.name)

#knowledge_graph

In [4]:
data = pd.read_csv("CSV_2020_sortiert.csv", sep=",")
entities = [entity for entity in data["relatedConceptsWork"]]
entities_distinct = list(dict.fromkeys(entities))

In [5]:
transformer = RDF2VecTransformer(
    Word2Vec(epochs=10),
    walkers=[RandomWalker(max_depth=4, max_walks=10)],
    verbose=1
)

In [6]:
embeddings, literals = transformer.fit_transform(knowledge_graph, entities_distinct)

100%|██████████| 2003/2003 [09:11<00:00,  3.63it/s]


Extracted 19807 walks for 2003 entities (553.5273s)
Fitted 19807 walks (0.6556s)


100%|██████████| 2003/2003 [00:00<00:00, 6578.01it/s]

Extracted 2003 literals for 2003 entities (0.3065s)


In [7]:
embeddings

In [8]:
literals